In [25]:
import os
import time
import hashlib
import json
from os import listdir
from os.path import isfile, join

import pandas as pd
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
from copy import deepcopy 
from sklearn.metrics import mean_absolute_error

import seaborn as sns 
import matplotlib.pyplot as plt

In [26]:
%matplotlib inline

plt.rcParams["figure.figsize"] = (14,8)
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2
plt.rcParams["axes.grid"] = True
plt.rcParams['axes.axisbelow'] = True

In [27]:
def get_data(sym):
    file_path = "./stockData/"
    csv_path = os.path.join(file_path, f"{sym}.csv")
    df = pd.read_csv(csv_path, parse_dates=True, index_col = ['Date'])
    df.drop('Symbol', axis=1, inplace=True)
    return df

In [28]:
def load_data_set(filename):
    dir_path = './stockData'
    
    with open(os.path.join(dir_path, filename), 'r') as f:
        results = json.load(f)
        
    return results

In [29]:
sp500_list = load_data_set('sp500_list.json')

In [34]:
len(sp500_list)

505

In [35]:
df_price = pd.DataFrame()

for sym in sp500_list:
    df = get_data(sym)
    df_price[sym] = df['Close']

In [37]:
df_price_corr = df_price.corr()

In [52]:
best_list = []

for _ in range(10):    
    for _ in range(10):
        corr_price = df_price_corr.copy()

        cor_dict = []

        rnd = []

        first = []

        for num in range(10):
            temp = np.random.randint(1,505 - (num * 51))
            rnd.append(temp)

        for r in rnd:
            i = corr_price.columns.values[r]
            first.append(i)

            temp = corr_price[i]
            temp_sort = temp.sort_values(ascending=False)
            cor_dict.append(temp_sort[1:31])

            idx = temp_sort[0:31].index.values
            corr_price.drop(idx, inplace=True)
            corr_price.drop(idx, axis=1, inplace=True)

        remain = list(corr_price.columns.values)
    
best_list.append(cor_dict)

In [53]:
scores = []

for co_list in best_list:
    t = []
    for cor in co_list:
        corr_mean = cor.mean()
        t.append(corr_mean)
    mean_score = np.mean(t)
    scores.append((mean_score))
    
print(np.max(scores))
best_idx = np.argmax(scores)
print(best_idx)

0.8618828975296552
0


In [54]:
a = best_list[best_idx]

for c in a:
    b = c.mean()
    print(b)

0.8467064506670184
0.8969440671599832
0.9751565196566973
0.8847584792649804
0.7754716792038144
0.7045023733731411
0.9399627582982931
0.8753836413722811
0.7681898967397685
0.9517531095605746


In [55]:
def save_exp_result(result):    
    filepath = './results'
    if not os.path.isdir(filepath):
        os.mkdir(filepath)

    filename = './results/dataset_2.json'
    with open(filename, 'w') as f:
        json.dump(result, f)

In [58]:
my_data_set = []

for c in a:
    temp = list(c.index.values)
    temp.append(c.name)
    my_data_set.append(temp)

In [59]:
save_exp_result(my_data_set)